In [3]:
from google.colab import files
uploaded = files.upload()

Saving Apple.jpeg to Apple.jpeg
Saving keras_model.h5 to keras_model.h5
Saving labels.txt to labels.txt


In [4]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Path image
image = Image.open("Apple.jpeg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

1/1 [==============================] - 1s 1s/step
Class: Apple
Confidence Score: 1.0


In [7]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
from google.colab import files
import io
from collections import defaultdict

model = load_model("keras_model.h5", compile=False)
class_names = [label.strip() for label in open("labels.txt", "r").readlines()]

uploaded = files.upload()

size = (224, 224)
total_images = 0
class_counts = defaultdict(int)
class_confidence_sum = defaultdict(float)

for filename in uploaded.keys():
    total_images += 1

    image = Image.open(io.BytesIO(uploaded[filename])).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    prediction = model.predict(data, verbose=0)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    class_counts[class_name] += 1
    class_confidence_sum[class_name] += confidence_score

    print(f"📷 Image: {filename}")
    print(f" → Predicted: {class_name.strip()}")
    print(f" → Confidence: {confidence_score*100:.2f}%")
    print("-" * 40)

print("\n✅ Summary:")
print("Total images processed:", total_images)
print("Expected class: Apple\n")

for class_name in class_names:
    count = class_counts[class_name]
    if count > 0:
        avg_conf = (class_confidence_sum[class_name] / count) * 100
        print(f"- {class_name.strip()}: {count} images, Avg Confidence: {round(avg_conf, 2)}%")
    else:
        print(f"- {class_name.strip()}: 0 images")

wrong_class = "Orange"
if any(wrong_class.lower() in k.lower() for k in class_counts):
    print(f"\n⚠️ Warning: {class_counts[wrong_class]} image(s) misclassified as {wrong_class}")

Saving Apple.jpeg to Apple (2).jpeg
Saving img_71.jpeg to img_71 (1).jpeg
Saving img_81.jpeg to img_81 (1).jpeg
Saving img_171.jpeg to img_171 (1).jpeg
Saving img_851.jpeg to img_851 (1).jpeg
Saving img_871.jpeg to img_871 (1).jpeg
Saving img_891.jpeg to img_891 (1).jpeg
Saving img_901.jpeg to img_901 (1).jpeg
📷 Image: Apple (2).jpeg
 → Predicted: 1 Apple
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_71 (1).jpeg
 → Predicted: 1 Apple
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_81 (1).jpeg
 → Predicted: 1 Apple
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_171 (1).jpeg
 → Predicted: 1 Apple
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_851 (1).jpeg
 → Predicted: 0 Orange
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_871 (1).jpeg
 → Predicted: 0 Orange
 → Confidence: 100.00%
----------------------------------------
📷 Image: img_891 (